In [71]:
from rdkit import Chem
import json
import pandas as pd

pdbcode = "5qj7"
pdbfile = open("../data/"+str(pdbcode)+".pdb").readlines()

hetatms  = []
conects = []

non_ligs = json.load(open("non_ligs", "r"))

for line in pdbfile:
    if line.startswith("HETATM"):
        hetatms.append(line)
    if line.startswith("CONECT"):
        conects.append(line)
           
final_hets = []
het_cons = []
for line in hetatms:
    if str(line.split()[3]) not in non_ligs:
        final_hets.append(line)

In [72]:
ligands = []

"""
Separate out the individual ligands 
"""
def find_ligand_names():
    for i in range(len(final_hets)):
        line_split = final_hets[i].split()
        LIG_NAME = '_'.join(map(str, line_split[3:6]))
        if LIG_NAME not in ligands:
            ligands.append(LIG_NAME)
        
  
def create_pdb_for_ligand(ligand):    
    ligands_connections = open("../resulting_data/"+str(pdbcode)+"_"+str(ligand)+".pdb", "w+")
    individual_ligand = []
    individual_ligand_conect = []
    code  = ligand.split('_')
    for atom in final_hets:
        atom_info = atom.split()
        atom_number = atom_info[1]
        if code[0] == atom_info[3] and code[1] == atom_info[4] and code[2] == atom_info[5]:
            individual_ligand.append(atom)
            for conection in conects:
                if atom_number in conection and conection not in individual_ligand_conect:
                    individual_ligand_conect.append(conection)

    ligand_het_con = individual_ligand+individual_ligand_conect

    for line in ligand_het_con:
        ligands_connections.write(str(line))

    ligands_connections.close()

    return (Chem.rdmolfiles.MolFromPDBFile("../resulting_data/"+str(pdbcode)+"_"+str(ligand)+".pdb"))
    
def create_mol_file(ligand, mol_obj):  
    return Chem.rdmolfiles.MolToMolFile(mol_obj, "../resulting_data/"+str(pdbcode)+"_"+str(ligand)+"_mol.mol")

    
find_ligand_names()
mol_1 = create_pdb_for_ligand(ligands[0])
mol_2 = create_pdb_for_ligand(ligands[1])
mol_3 = create_pdb_for_ligand(ligands[2])
mol_4 = create_pdb_for_ligand(ligands[3])
hello = create_mol_file(ligands[0], mol_1)
create_mol_file(ligands[1], mol_2)
create_mol_file(ligands[2], mol_3)
create_mol_file(ligands[3], mol_4)

In [73]:
writer = Chem.rdmolfiles.SDWriter('../resulting_data/out.sdf')
writer.write(mol_1)
writer.write(mol_2)
writer.write(mol_3)
writer.write(mol_4)